In [1]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 3.7 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.7.0-py3-none-any.whl size=243940 sha256=4cf16ad0b5758f1399abe0f66e09787047f8eefc3d23aa3e00ba51431f027087
  Stored in directory: /Users/dbabichenko/Library/Caches/pip/wheels/6d/be/1e/749b84bba4b7429145ea09ccb46114fab4748625eb319f286a
Successfully built neo4j


In [50]:
from neo4j import GraphDatabase
from uuid import uuid4
import pandas as pd

In [61]:
# The following class is based on the "Create a graph database in Neo4j using Python" 
# tutorial by CJ Sullivan, Feb 10, 2021. 
# https://towardsdatascience.com/create-a-graph-database-in-neo4j-using-python-4172d40f89c4


class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
    
    def query_to_dataframe(self, query, parameters=None, db=None):
        df = pd.DataFrame([dict(_) for _ in self.query(query, parameters, db)])
        return df


In [62]:
conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="neo4j",              
                       pwd="admin123")

In [63]:
qry = "CREATE (f:Flight) SET f.flight_id = '" + str(uuid4()) + "' RETURN f;"
#print(qry)
result = conn.query(qry)
print(result)

[<Record f=<Node element_id='4:e8ca3093-9c89-424c-8663-20395c2c8b7b:5' labels=frozenset({'Flight'}) properties={'flight_id': '250e0ce5-13a6-4688-a447-1baf9053aece'}>>]


In [64]:
qry = "MATCH (f:Flight) RETURN f.flight_id;"
result = conn.query(qry)
print(result)

[<Record f.flight_id='c53e8880-4cb6-4c28-9af5-bb2110289cbf'>, <Record f.flight_id='4c10f0a4-bd23-4ccd-bf46-c775b118c412'>, <Record f.flight_id='16ff5c0d-69dc-42a9-b166-571bd52d69d9'>, <Record f.flight_id='0fe2a462-478e-4a51-bd63-fa959823c496'>, <Record f.flight_id='0fe2a462-478e-4a51-bd63-fa959823c496'>, <Record f.flight_id='250e0ce5-13a6-4688-a447-1baf9053aece'>]


In [65]:
for node in result:
    print(node[0])

c53e8880-4cb6-4c28-9af5-bb2110289cbf
4c10f0a4-bd23-4ccd-bf46-c775b118c412
16ff5c0d-69dc-42a9-b166-571bd52d69d9
0fe2a462-478e-4a51-bd63-fa959823c496
0fe2a462-478e-4a51-bd63-fa959823c496
250e0ce5-13a6-4688-a447-1baf9053aece


In [66]:
df = pd.DataFrame([dict(_) for _ in conn.query(qry)])
df.head(20)

,f.flight_id
0,c53e8880-4cb6-4c28-9af5-bb2110289cbf
1,4c10f0a4-bd23-4ccd-bf46-c775b118c412
2,16ff5c0d-69dc-42a9-b166-571bd52d69d9
3,0fe2a462-478e-4a51-bd63-fa959823c496
4,0fe2a462-478e-4a51-bd63-fa959823c496
5,250e0ce5-13a6-4688-a447-1baf9053aece


In [67]:
df = conn.query_to_dataframe(qry)

In [68]:
df.head()

,f.flight_id
0,c53e8880-4cb6-4c28-9af5-bb2110289cbf
1,4c10f0a4-bd23-4ccd-bf46-c775b118c412
2,16ff5c0d-69dc-42a9-b166-571bd52d69d9
3,0fe2a462-478e-4a51-bd63-fa959823c496
4,0fe2a462-478e-4a51-bd63-fa959823c496
